In [60]:
import numpy as np 
import pandas as pd
import torch

In [61]:
cv=torch.load('dataset/course_nm_tensor.pt')

In [62]:
cvnpy=cv.detach().numpy()
cvnpy[:4]

array([[ 0.00144092, -0.01703043,  0.00412345, ...,  0.0865251 ,
        -0.05929217,  0.00758869],
       [-0.00358773, -0.01746405, -0.12736031, ...,  0.02512024,
        -0.01919973,  0.01910394],
       [-0.00345605, -0.03809951, -0.0692881 , ...,  0.03877716,
        -0.03475577, -0.03682799],
       [ 0.0022921 , -0.07450938, -0.00588197, ...,  0.00472871,
        -0.03109727, -0.00817915]], dtype=float32)

In [63]:
#group by whether it is 전공 
#같은 전공 내에서 진행

curridb=pd.read_csv('original_data/curriculumDB.txt')
coursedb=pd.read_csv('original_data/courseDB.txt')
ccdb=pd.read_csv('original_data/curri_courseDB.txt')

In [64]:
curridb.head(5)

,curr_id,the_year,subject_name,major_division,elec_num,ge_info
0,628b75c1920a8302ce7ea449535834a877720b79499426...,2019,건축도시시스템공학,1,138,10
1,e3e972226e56ed375c91a5f95e8c7f098e7bd3bfbac3e9...,2019,건축학,1,148,10
2,2ec8f0ea892df31db7f724bb6f5905e5a32a524ee7d33f...,2019,기후·에너지시스템공학,1,138,10
3,b4a6f11c5c2fdf083a2cc098db210bd7a91c1357a1164e...,2019,사이버보안,1,139,10
4,07902b1e387b5dcdd2ae8d9929576ac23aacd48c628d3a...,2019,식품공학,1,138,10


예시: 건축도시시스템공학,건축학,사이버보안

In [65]:
type(curridb.loc[0,'the_year'])
curridb.loc[0,'the_year']==2019
curridb.loc[3,'subject_name'].strip()=='사이버보안'
#print(curridb.loc[3,'subject_name'])

True

In [66]:
#subject_name, the_year정보를 바탕으로 구한다
subject_name='사이버보안'
the_year=2019

curr_id=curridb[(curridb['the_year']==the_year) & (curridb['subject_name'].apply(str.strip)==subject_name)]['curr_id']
curr_id

3    b4a6f11c5c2fdf083a2cc098db210bd7a91c1357a1164e...
Name: curr_id, dtype: object

In [67]:
ccdb.head(5)
type(ccdb['curr_id'].values)

np.array(ccdb.loc[0,'curr_id'])
str(curr_id.values[0])

'b4a6f11c5c2fdf083a2cc098db210bd7a91c1357a1164e01abae4e28d845b21a'

In [69]:
cvnpy.shape

(1035, 768)

In [164]:
from sklearn.metrics.pairwise import cosine_similarity
class Node:
    def __init__(self,course_id,standard,curr_id,course_vec):
        self.course_id=course_id 
        self.standard=standard 
        self.curr_id=curr_id 
        self.course_vec=course_vec

class RankMatrix:
    def __init__(self,nodes):
        self.nodes=nodes
        self.similarity_matrix=None
        self.run_cosine_similarity() 
        
    def run_cosine_similarity(self):
        course_vec=[node.course_vec for node in self.nodes] 
        self.similarity_matrix=cosine_similarity(course_vec,course_vec)
        
    def retrieve_similarity(self,course_id):
        course_ids=[i.course_id for i in self.nodes]
        course_standards=[i.standard for i in self.nodes]
        index_course=course_ids.index(course_id)
        return self.similarity_matrix[index_course],course_ids,course_standards
    
    
    def retrieve_top_10(self,course_ids,filter=''):
        #make it a node 
        def create_rank_node(course_id,sim,ccourse_id,stds):#ccourse_id는 비교 대상, course_id는 이번에 비교되는 애임 sim은 둘 사이 유사도임
            return {'course_id':course_id,'sim':sim,'ccourse_id':ccourse_id,'stds':stds}
        
        sim_list=[]
        for course in course_ids:
            sims,ids,stds=self.retrieve_similarity(course)
            for similarity,cid,std in zip(sims,ids,stds):
                if course==cid:
                    continue
                sim_list.append(create_rank_node(cid,similarity,course,std))
        
        result=sim_list.copy()

        if filter=='전공':
            for node in result:
                if node['stds'].find('전공')==-1:#없으면
                    result.pop(result.index(node))
        elif filter=='융복합':
            for node in result:
                if node['stds'].find('융복합')==-1:#없으면
                    result.pop(result.index(node))
        elif filter=='교양':
            for node in result:
                if node['stds'].find('전공')!=-1:#있으면
                    result.pop(result.index(node))
                if node['stds'].find('융복합')!=-1:#있으면
                    result.pop(result.index(node))
        
        
        result=merge_sort(result)
        
        #중복 제거
        visited=[]
        result_cpy=result.copy()
        for node in result:
            if node['course_id'] in visited:
                result_cpy.pop(result_cpy.index(node))
            else:
                visited.append(node['course_id'])
        return result_cpy[-10:]
    
r=RankMatrix([Node('1938','standard','dkfjsl',vec) for vec in np_1])
r.retrieve_similarity('1938')

(array([1.        , 0.39910215]), ['1938', '1938'], ['standard', 'standard'])

In [163]:
a='asdkfh'
a.find('z')

-1

In [71]:
def merge(left, right):
    if len(left) == 0:
        return right

    if len(right) == 0:
        return left

    result = []
    index_left = index_right = 0

    while len(result) < len(left) + len(right):
        
        if left[index_left]['sim'] <= right[index_right]['sim']:
            result.append(left[index_left])
            index_left += 1
        else:
            result.append(right[index_right])
            index_right += 1

        # If you reach the end of either array, then you can
        # add the remaining elements from the other array to
        # the result and break the loop
        if index_right == len(right):
            result += left[index_left:]
            break

        if index_left == len(left):
            result += right[index_right:]
            break

    return result
def merge_sort(array):
    
    if len(array) < 2:
        return array

    midpoint = len(array) // 2
    
    return merge(
        left=merge_sort(array[:midpoint]),
        right=merge_sort(array[midpoint:]))

In [72]:
#currid를 바탕으로 사용자의 전공과목을 구한다 
#cold start 
#사용자의 전공과목과 전부 인접행렬을 만들고 
#해당 과목 인접행렬에서 가장 높은 것 N개를 구함


all_c=ccdb[ccdb['curr_id']==str(curr_id.values[0])]['course_id'].values

#종류 식별
st_cDB=pd.read_csv('original_data/standard_courseDB.txt')

#모든 학수번호마다 종류 파악
node_list=[] #하나의 교과과정표에 있는 과목의 리스트이다.
course_list=list(coursedb['course_id'])
for c in all_c:
    st_cDB_curr=st_cDB[st_cDB['curr_id']==str(curr_id.values[0])] #사용자의 교과과정표 내 수업에 대해
    st_cDB_cc=st_cDB_curr[st_cDB_curr['course_id']==c ]['standard_name'].values[0] #standard이름 파악
    index=course_list.index(c)#courseDB내 위치 파악해서 벡터와 연결
    node_list.append(Node(c,st_cDB_cc,str(curr_id.values[0]),cvnpy[index]))#course_id, standard, vector 로 된 노드를 만듦 

## test: 그냥 아무 노드 10개를 가지고 시도 

In [168]:
node_list[10].course_id

'11033'

In [170]:
r=RankMatrix(node_list)
k=r.retrieve_top_10([i.course_id for i in node_list[:10]])

In [172]:
recommender_analyze(all_curri_for_c,k)

,type,course_id_from,course_id_to,subject_name,course_name_from,course_name_to,credit_to,prev
0,전공,11094,11073,건축도시시스템공학,고전읽기와글쓰기,프랑스어Ⅰ,2.0,None
1,전공,11094,11368,건축도시시스템공학,고전읽기와글쓰기,정보와인공지능의수리적이해,3.0,None
2,전공,11094,10135,건축도시시스템공학,고전읽기와글쓰기,지구의역사,3.0,None
3,전공,11094,38509,건축도시시스템공학,고전읽기와글쓰기,사이버보안중기인턴십III,3.0,None
4,전공,11094,11205,건축도시시스템공학,고전읽기와글쓰기,컴퓨터과학적인식과문제해결,3.0,None
5,전공,11094,37842,건축도시시스템공학,고전읽기와글쓰기,"캄보디아교육,환경,문화의이해",3.0,None
6,전공,11094,38476,건축도시시스템공학,고전읽기와글쓰기,컴퓨터시스템,3.0,None
7,전공,11253,38513,건축도시시스템공학,나눔의인간학,사이버보안글로벌필드트레이닝I,1.0,None
8,전공,11094,38506,건축도시시스템공학,고전읽기와글쓰기,사이버보안단기인턴십III,1.0,None
9,전공,11253,11309,건축도시시스템공학,나눔의인간학,문제해결과SW프로그래밍,3.0,None


## st_cDB를 가지고 이제 종류별로 가져오기

In [156]:
j=[]
y=[]
g=[]
for node in node_list:
    if node.standard.find('전공')!=-1:
        j.append(node.course_id)
    elif node.standard.find('융복합')!=-1:
        y.append(node.course_id)
    else:
        g.append(node.course_id)
j[:3]

['38411', '38403', '20410']

In [180]:
#전공, 융복합, 교양에 대해 각각 10개만 뽑는 코드 짜기 
r=RankMatrix(node_list)

rcj=r.retrieve_top_10(j,filter='전공')
rcy=r.retrieve_top_10(y,filter='융복합')
rcg=r.retrieve_top_10(g,filter='교양')
rcj2=r.retrieve_top_10(j,filter='d')
rcy2=r.retrieve_top_10(y,filter='d')
rcg2=r.retrieve_top_10(g,filter='d')
rcy

[]

# 여러개 전공에 대해서 추천하기

In [139]:
class Recommender:
    def __init__(self,subjects,years):
        self.subjects=subjects
        self.years=years 
        #sanity check
        if len(subjects)!=len(years):
            Exception('You need to assign years per subject areas')
        self.all_curri_for_c=self.get_curri_of_subjects()
        
    def get_curri_of_subjects(self):
        
        #전체 데이터 호출
        curridb=pd.read_csv('original_data/curriculumDB.txt')
        ccdb=pd.read_csv('original_data/curri_courseDB2.txt')
        st_cDB=pd.read_csv('original_data/standard_courseDB.txt')
        coursedb=pd.read_csv('original_data/courseDB.txt')  
        cv=torch.load('dataset/course_nm_tensor.pt')
        cvnpy=cv.detach().numpy()
        
        #일부 데이터 수집
        curr_ids=[]
        for subject_name,the_year in zip(self.subjects,self.years):
            curr_id=curridb[(curridb['the_year']==the_year) & (curridb['subject_name'].apply(str.strip)==subject_name)]['curr_id'].values[0]
            curr_ids.append(curr_id)
            
        all_curri_for_c=pd.concat([ccdb[ccdb['curr_id']==str(curr_id)]for curr_id in curr_ids])[['course_id','curr_id']]#course에 대해 curri_id
        all_c=pd.concat([ccdb[ccdb['curr_id']==str(curr_id)]for curr_id in curr_ids])['course_id'].values 
        
        
        #모든 학수번호마다 종류 파악
        node_list=[] #하나의 교과과정표에 있는 과목의 리스트이다.
        course_list=list(coursedb['course_id'])
        st_cDB_curr=pd.concat([st_cDB[st_cDB['curr_id']==str(ci)] for ci in curr_ids]) #사용자의 교과과정표 내 수업에 대해
   

        for checker_count,c in enumerate(all_c):
            if checker_count%100==0:
                print(checker_count,'만큼 왔다')
            st_cDB_cc=st_cDB_curr[st_cDB_curr['course_id']==c ]['standard_name'].values[0] #standard이름 파악
            try:
                index=course_list.index(c)#courseDB내 위치 파악해서 벡터와 연결
                node_list.append(Node(c,st_cDB_cc,str(curr_id),cvnpy[index]))
            except:
                print(c)
        self.node_list=node_list
        print('node_list done!')
        
        
        del curridb,ccdb,st_cDB,coursedb,cv
        return all_curri_for_c

    def recommend(self):
        #추천의 입력 필터링하기
        j=[]
        y=[]
        g=[]
        for node in self.node_list:
            if node.standard.find('전공')!=-1:
                j.append(node.course_id)
            elif node.standard.find('융복합')!=-1:
                y.append(node.course_id)
            else:
                g.append(node.course_id)
        
        #추천 찾기      
        r=RankMatrix(self.node_list)

        #추천 결과 필터링해서 받기
        print('추천 for 전공')
        rcj=r.retrieve_top_10(j,filter='전공')
        print('추천 for 융복합')
        rcy=r.retrieve_top_10(y,filter='융복합')
        print('추천 for 교양')
        rcg=r.retrieve_top_10(g,filter='교양')
        
        return rcj,rcy,rcg

In [182]:
rr=Recommender(['건축도시시스템공학','건축학','사이버보안'],[2019,2019,2019])


0 만큼 왔다
100 만큼 왔다
200 만큼 왔다
300 만큼 왔다
400 만큼 왔다
500 만큼 왔다
600 만큼 왔다
node_list done!


In [184]:
rcj3,rcy3,rcg3=rr.recommend()
rcg3

추천 for 전공
추천 for 융복합
추천 for 교양


[{'course_id': '38503',
  'sim': 0.88019377,
  'ccourse_id': '11289',
  'stds': '전공선택'},
 {'course_id': '38491',
  'sim': 0.8836429,
  'ccourse_id': '11289',
  'stds': '전공필수'},
 {'course_id': '38507',
  'sim': 0.8841116,
  'ccourse_id': '11289',
  'stds': '전공선택'},
 {'course_id': '38487',
  'sim': 0.8853723,
  'ccourse_id': '11289',
  'stds': '전공선택'},
 {'course_id': '38483',
  'sim': 0.8881506,
  'ccourse_id': '11289',
  'stds': '전공선택'},
 {'course_id': '38919',
  'sim': 0.89225066,
  'ccourse_id': '11289',
  'stds': '전공선택'},
 {'course_id': '38481',
  'sim': 0.89274365,
  'ccourse_id': '11289',
  'stds': '전공선택'},
 {'course_id': '38489',
  'sim': 0.89593065,
  'ccourse_id': '11289',
  'stds': '전공선택'},
 {'course_id': '38511',
  'sim': 0.902614,
  'ccourse_id': '11289',
  'stds': '전공선택'},
 {'course_id': '*39184',
  'sim': 0.9674399,
  'ccourse_id': '11289',
  'stds': '전공선택'}]

In [147]:
rcy

[{'course_id': '11111',
  'sim': -0.062258378,
  'ccourse_id': '10887',
  'stds': '융복합교양(과학과기술)'},
 {'course_id': '11234',
  'sim': -0.05740011,
  'ccourse_id': '10887',
  'stds': '융합기초'},
 {'course_id': '36339',
  'sim': -0.0523066,
  'ccourse_id': '10181',
  'stds': '전공기초(선택필1)'},
 {'course_id': '38462',
  'sim': -0.0398903,
  'ccourse_id': '10181',
  'stds': '전공선택'},
 {'course_id': '36365',
  'sim': -0.039349742,
  'ccourse_id': '10887',
  'stds': '전공선택'},
 {'course_id': '38088',
  'sim': -0.020799201,
  'ccourse_id': '11105',
  'stds': '전공선택'},
 {'course_id': '11110',
  'sim': -0.020014826,
  'ccourse_id': '10887',
  'stds': '융복합교양(표현과예술)'},
 {'course_id': '36364',
  'sim': -0.01305346,
  'ccourse_id': '10887',
  'stds': '전공선택'},
 {'course_id': '38442',
  'sim': -0.0010998193,
  'ccourse_id': '10887',
  'stds': '전공선택'},
 {'course_id': '20412',
  'sim': 0.00027134828,
  'ccourse_id': '10887',
  'stds': '전공기초(선택필1)'}]

In [114]:
all_curri_for_c=r.get_curri_of_subjects()

0 만큼 왔다
100 만큼 왔다
200 만큼 왔다
300 만큼 왔다
400 만큼 왔다
500 만큼 왔다
600 만큼 왔다
node_list done!


In [115]:
all_curri_for_c.head(5)

,course_id,curr_id
0,10023,628b75c1920a8302ce7ea449535834a877720b79499426...
1,11256,628b75c1920a8302ce7ea449535834a877720b79499426...
2,11254,628b75c1920a8302ce7ea449535834a877720b79499426...
3,11095,628b75c1920a8302ce7ea449535834a877720b79499426...
4,11253,628b75c1920a8302ce7ea449535834a877720b79499426...


In [144]:
#DB에 넣기
def recommender_fit_DB(all_curri_for_c,li,ty='전공'):
    coursedb=pd.read_csv('original_data/courseDB.txt')  
    prevdb=pd.read_csv('original_data/prev_listDB.txt')  
    currdb=pd.read_csv('original_data/curriculumDB.txt') 
    db=[] 
    for node in li:
        #정보 1
        from_=node['ccourse_id']
        #정보 2
        to_=node['course_id']
        #정보 3,4
        credit,course_name=coursedb[coursedb['course_id']==to_].loc[:,['credit','course_name']].values[0]#결과의 정보 
        
        #정보 5
        prevs=prevdb[prevdb['course_id']==to_]
        if len(prevs)>0:
            prevs=prevs.values()
            prev_result=[]
            for prev in prevs:
                tmp=coursedb[coursedb['course_id']==prev].loc[:,'course_name']
                if tmp:
                    prev_result.append()
                    
            if len(prev_result)==0:
                prev_result='None'
        else:
            prevs_result='None'
        
        #정보 6
        curr_id=all_curri_for_c[all_curri_for_c['course_id']==from_]['curr_id'].values[0]
        subject_name=currdb[currdb['curr_id']==curr_id]['subject_name'].values[0]
        db.append({'type':ty,'course_id_from':from_,'course_id_to':to_,'subject_name':subject_name,'course_name':course_name,'credit':credit,'prev':prevs_result})
    return pd.DataFrame(db,columns=['type','course_id_from','course_id_to','subject_name','course_name','credit','prev'])

rcj_rendered=recommender_fit_DB(all_curri_for_c,rcj,ty='전공')
rcy_rendered=recommender_fit_DB(all_curri_for_c,rcy,ty='융복합')
rcg_rendered=recommender_fit_DB(all_curri_for_c,rcg,ty='교양')

In [145]:
recommender_result=pd.concat([rcj_rendered,rcy_rendered,rcg_rendered],axis=0)
recommender_result.to_csv('recommender_data/recommend_no_duplicate.txt',header=None,index=False)

In [ ]:
rcm_cpy=recommender_result.copy()
def type_to_num(element):
    if element=='전공':
        return 3
    elif element=='융복합':
        return 2
    elif element=='교양':
        return 1

rcm_cpy['type']=rcm_cpy['type'].apply(type_to_num)

In [ ]:
rcm_cpy.to_csv('recommender_data/recommend_type_to_num.txt',header=None)

In [135]:
recommender_result

,type,course_id_from,course_id_to,subject_name,course_name,credit,prev
0,전공,38395,38396,건축도시시스템공학,건축도시시스템글로벌필드트레이닝II,1.0,None
1,전공,38396,38395,건축도시시스템공학,건축도시시스템글로벌필드트레이닝I,1.0,None
2,전공,38405,36345,건축학,건축설계2,6.0,None
3,전공,36345,38405,건축학,건축컴퓨터프로그래밍및실습,3.0,None
4,전공,38487,38484,사이버보안,빅데이터보안,3.0,None
5,전공,38484,38487,사이버보안,블록체인응용,3.0,None
6,전공,38466,38494,건축학,사이버법률과정책,3.0,None
7,전공,38494,38466,사이버보안,건축융합설계,3.0,None
8,전공,37284,30002,건축학,일반물리학II,3.0,None
9,전공,30002,37284,사이버보안,건축시공시스템,3.0,None


# 추천시스템 성능 평가하기

In [173]:
#DB에 넣기
def recommender_analyze(all_curri_for_c,li,ty='전공'):
    coursedb=pd.read_csv('original_data/courseDB.txt')  
    prevdb=pd.read_csv('original_data/prev_listDB.txt')  
    currdb=pd.read_csv('original_data/curriculumDB.txt') 
    db=[] 
    for node in li:
        #정보 1
        from_=node['ccourse_id']
        #정보 2
        to_=node['course_id']
        #정보 3,4
        credit_from,course_name_from=coursedb[coursedb['course_id']==from_].loc[:,['credit','course_name']].values[0]#결과의 정보 
        
        credit_to,course_name_to=coursedb[coursedb['course_id']==to_].loc[:,['credit','course_name']].values[0]#결과의 정보 
        
        #정보 5
        prevs=prevdb[prevdb['course_id']==to_]
        if len(prevs)>0:
            prevs=prevs.values()
            prev_result=[]
            for prev in prevs:
                tmp=coursedb[coursedb['course_id']==prev].loc[:,'course_name']
                if tmp:
                    prev_result.append()
                    
            if len(prev_result)==0:
                prev_result='None'
        else:
            prevs_result='None'
        
        #정보 6
        curr_id=all_curri_for_c[all_curri_for_c['course_id']==from_]['curr_id'].values[0]
        subject_name=currdb[currdb['curr_id']==curr_id]['subject_name'].values[0]
        sims=node['sim']
        db.append({'type':ty,'course_id_from':from_,'course_id_to':to_,'course_name_from':course_name_from,'course_name_to':course_name_to,'sim':sims})
    return pd.DataFrame(db,columns=['type','course_id_from','course_id_to','course_name_from','course_name_to','sim'])

rcj_rendered2=recommender_analyze(all_curri_for_c,rcj2,ty='전공')
rcy_rendered2=recommender_analyze(all_curri_for_c,rcy2,ty='융복합')
rcg_rendered2=recommender_analyze(all_curri_for_c,rcg2,ty='교양')

In [185]:
rcj_rendered3=recommender_analyze(all_curri_for_c,rcj3,ty='전공')
rcy_rendered3=recommender_analyze(all_curri_for_c,rcy3,ty='융복합')
rcg_rendered3=recommender_analyze(all_curri_for_c,rcg3,ty='교양')

In [188]:
rcg_rendered3.sort_values(by='sim',ascending=False)

,type,course_id_from,course_id_to,course_name_from,course_name_to,sim
9,교양,11289,*39184,메이커시대의디자인씽킹,사이버보안산학실무설계 IV,0.967440
8,교양,11289,38511,메이커시대의디자인씽킹,사이버보안자기설계학습II,0.902614
7,교양,11289,38489,메이커시대의디자인씽킹,사이버물리시스템보안,0.895931
6,교양,11289,38481,메이커시대의디자인씽킹,정보보호프로토콜,0.892744
5,교양,11289,38919,메이커시대의디자인씽킹,알고리즘과계산복잡도,0.892251
4,교양,11289,38483,메이커시대의디자인씽킹,네트워크보안,0.888151
3,교양,11289,38487,메이커시대의디자인씽킹,블록체인응용,0.885372
2,교양,11289,38507,메이커시대의디자인씽킹,사이버보안중기인턴십I,0.884112
1,교양,11289,38491,메이커시대의디자인씽킹,사이버보안종합설계프로젝트 I,0.883643
0,교양,11289,38503,메이커시대의디자인씽킹,사이버보안산학실무설계III,0.880194


In [187]:
rcj_rendered3.sort_values(by='sim',ascending=False)

,type,course_id_from,course_id_to,course_name_from,course_name_to,sim
9,전공,36348,20477,건축설계5,운영체제,0.086346
8,전공,36348,36387,건축설계5,서양건축사,0.070426
7,전공,36348,11102,건축설계5,"에너지의이해,이용,실천",0.064496
6,전공,36386,37418,건축프로젝트관리,친환경건축시스템,0.056041
5,전공,36386,11234,건축프로젝트관리,컴퓨터적사고와조형적활용,0.049343
4,전공,36386,10910,건축프로젝트관리,위험판단심리학,0.043407
3,전공,36386,36358,건축프로젝트관리,도시설계,0.043051
2,전공,36386,11111,건축프로젝트관리,SF영화로배우는철학,0.015306
1,전공,36386,20215,건축프로젝트관리,예술철학,0.009035
0,전공,36348,11035,건축설계5,"퍼포먼스인문학: 제의,놀이,연극",0.001226


In [186]:
rcy_rendered3.sort_values(by='sim',ascending=False)

,type,course_id_from,course_id_to,course_name_from,course_name_to,sim
9,융복합,10887,20412,현대중국의사회와문화,일반생물학,0.000271
8,융복합,10887,38442,현대중국의사회와문화,도시계획및관리,-0.001100
7,융복합,10887,36364,현대중국의사회와문화,건축과도시,-0.013053
6,융복합,10887,11110,현대중국의사회와문화,예술가와그의시대,-0.020015
5,융복합,11105,38088,직업세계와법,건축도시시스템자기설계프로젝트IV,-0.020799
4,융복합,10887,36365,현대중국의사회와문화,공간구성과분석,-0.039350
3,융복합,10181,38462,환경과인간,건축자기설계프로젝트I,-0.039890
2,융복합,10887,11234,현대중국의사회와문화,컴퓨터적사고와조형적활용,-0.057400
1,융복합,10887,11111,현대중국의사회와문화,SF영화로배우는철학,-0.062258
0,융복합,10887,10910,현대중국의사회와문화,위험판단심리학,-0.089925


In [178]:
rcy_rendered2.sort_values(by='sim',ascending=False)

,type,course_id_from,course_id_to,course_name_from,course_name_to,sim


In [176]:
rcj_rendered2.sort_values(by='sim',ascending=False)

,type,course_id_from,course_id_to,course_name_from,course_name_to,sim
9,전공,38499,38506,모바일보안,사이버보안단기인턴십III,0.397848
8,전공,38497,11309,디지털포렌식스,문제해결과SW프로그래밍,0.389218
7,전공,38497,38513,디지털포렌식스,사이버보안글로벌필드트레이닝I,0.378409
6,전공,38499,38476,모바일보안,컴퓨터시스템,0.362158
5,전공,38499,37842,모바일보안,"캄보디아교육,환경,문화의이해",0.361666
4,전공,38499,11205,모바일보안,컴퓨터과학적인식과문제해결,0.333077
3,전공,38499,38509,모바일보안,사이버보안중기인턴십III,0.295284
2,전공,38499,11368,모바일보안,정보와인공지능의수리적이해,0.285698
1,전공,38494,20416,사이버법률과정책,일반화학I,0.278363
0,전공,38499,10135,모바일보안,지구의역사,0.274536


In [148]:

def average_sim(nodes):
    sims=[]
    for node in nodes:
        sims.append(node['sim'])
    return sum(sims)/len(sims)

print(average_sim(rcj),average_sim(rcy),average_sim(rcg))

0.04798471499234438 -0.030590108968317507 -0.03728255787864328
